In [106]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
import asyncio
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
from sklearn.metrics import normalized_mutual_info_score

load_dotenv()
GEMINI_API_KEY = os.getenv("KEY")

# text is regarded as key
seven_newsgroups_data = pd.read_csv('fetch_7newsgroups.csv')

def split_train_test(data):
    train, test = train_test_split(data, test_size=0.2)

    train = train.sort_values(by='label')
    train_first_doc = train.groupby('label').first().reset_index()

    train_first_doc['div'] = 'train'
    test['div'] = 'test'

    return test, train_first_doc

test_df, instruction_df = split_train_test(seven_newsgroups_data)
test_df["name"] = range(1, len(test_df) + 1)
print(test_df.iloc[5])
print("test Length: ",len(test_df))
print("instruction Length: ", len(instruction_df))
print("original length: ", len(seven_newsgroups_data))


Unnamed: 0                                                    7185
text             From: pmetzger@snark.shearson.com (Perry E. Me...
label20                                                         11
labels_name20                                            sci.crypt
label                                                      Science
div                                                           test
name                                                             6
Name: 7185, dtype: object
test Length:  3770
instruction Length:  7
original length:  18846


In [117]:
def make_prompts(bbc_instructions, bbc_data):
    prompts = []
    
    # General instructions and fixed texts
    general_instruction = (
        "You are a perfect topic modeling machine. Given a text and the different topics, "
        "you will classify the texts to the correct topic. First you will receive the topics, "
        "afterwards an example and finally the text you have to assign one of the before mentioned topics to."
    )
    topics = "The topics are sport, technology, politics, vehicles, religion, miscellaneous and science. Please make sure, you know the topics and their meaning."
    transition_to_examples = "Now an example for each of the categories will follow."
    transition_to_text_to_classify = (
        "Now the text, you have to classify will follow. Please assess its topic and answer only the topic of it."
    )

    # Iterate through the test DataFrame rows
    for _, test_row in bbc_data.iterrows():
        prompt = general_instruction + "\n" + topics + "\n" + transition_to_examples + "\n"

        # Iterate through instruction DataFrame to add examples
        for _, instruction_row in bbc_instructions.iterrows():
            category = instruction_row['label']
            example_text = instruction_row['text']
            prompt += f"For the following text: \n{example_text}\nThe correct answer would be: {category}\n"

        # Add the actual text to classify from the test set
        text_to_classify = test_row['text']
        prompt += transition_to_text_to_classify + "\n" + text_to_classify + "\n"
        name = f"{test_row['name']}"
        #print(name)
        prompt_dict = {}
        prompt_dict[name] = prompt
        prompts.append(prompt_dict)
    return prompts            

prompts = make_prompts(instruction_df, test_df)
prompts = prompts[2190:]
print(prompts[0])
print(len(prompts))

{'2191': 'You are a perfect topic modeling machine. Given a text and the different topics, you will classify the texts to the correct topic. First you will receive the topics, afterwards an example and finally the text you have to assign one of the before mentioned topics to.\nThe topics are sport, technology, politics, vehicles, religion, miscellaneous and science. Please make sure, you know the topics and their meaning.\nNow an example for each of the categories will follow.\nFor the following text: \nFrom: glazier@isr.harvard.edu (Andrew Baker Glazier)\nSubject: Re: HEAVY METAL (the magazine) for sale (NOT the MUSIC)\nArticle-I.D.: burrhus.1993Apr6.151840.3953\nOrganization: Harvard University, Cambridge, MA\nLines: 23\n\nIn article <1prs2t$eqp@transfer.stratus.com> Arthur_Noguerola@vos.stratus.com writes:\n>\n>          I  am  cleaning  out  the coffers.  I have a virtually \n>          MINT collection of HEAVY METAL magazine.  This is  NOT \n>          a  music  mag  but the reall

In [118]:


async def prompt_llm(prompt):
    semaphore = asyncio.Semaphore(5)
    async with semaphore:
        name = list(prompt.keys())[0]
        promptAI = prompt[name]   

        generation_config = {
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 64,
        "max_output_tokens": 8192,
        "response_mime_type": "text/plain",
        }
        model = genai.GenerativeModel(
        model_name="gemini-1.5-pro",
        generation_config=generation_config,
        safety_settings = {HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE},
        )
        
        chat_session = model.start_chat(
        history=[])
        answer = chat_session.send_message(promptAI)
        text_response = answer._result.candidates[0].content.parts[0].text
        
        return {name : text_response}

In [122]:
genai.configure(api_key=GEMINI_API_KEY)

async def prompt_orchestrator():
      batch_size = 5
      avatiables = []
      batches = [prompts[i:i + batch_size] for i in range(0, len(prompts), batch_size)]

      # Ensure the last batch is smaller if it's less than batch_size
      if len(batches[-1]) < batch_size:
            remaining = len(batches[-1])
            batches[-1] = prompts[-remaining:]

      # If the last batch is part of the earlier slices and it's less than batch_size
      if len(prompts) % batch_size != 0:
            remaining = len(prompts) % batch_size
            batches[-1] = prompts[-remaining:]

      try:
            for batch in batches:
                  avatiables_batch = await asyncio.gather(*(prompt_llm(prompt) for prompt in batch))
                  avatiables.extend(avatiables_batch)
                  print("sleeping")
                  await asyncio.sleep(1)

      except Exception as e:
            print(f"An error occurred: {e}. Returning the accumulated results.")
            return avatiables
      
      return avatiables

results = await prompt_orchestrator()

print(results)


An error occurred: block_reason: OTHER
. Returning the accumulated results.
[]


In [112]:
def add_correct_anser(results, test_df):
    result_with_correct_answer = []
    for result in results:
        for key, value in result.items():
            value = value.replace("\n", "")
            value = value.replace(" \n", "")
            value = value.replace(" ","")
            value = value.replace("*","")
            value = value.lower()
            matching_rows = test_df.loc[test_df['name'] == int(key), 'label']
            if not matching_rows.empty:
                if type(matching_rows.values[0]) == float: 
                    print(f"Nan value", matching_rows.values[0])
                else:
                    category_value = matching_rows.values[0]
                    category_value = category_value.lower()
                    result_with_correct_answer.append({key: (value, category_value)})
            else:
                print(f"No matching category found for key: {key}")
                result_with_correct_answer.append({key: (value, None)})
    
    return result_with_correct_answer
results_with_correct_answer = add_correct_anser(results, test_df)
print(len(results_with_correct_answer))
print(results_with_correct_answer)

Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan value nan
Nan va

In [113]:
def extract_ground_truth_and_predictions(results_with_correct_answer):
    ground_truth = []
    predictions = []
    for result in results_with_correct_answer:
        for key, value in result.items():
            ground_truth.append(value[1])
            predictions.append(value[0])
    return ground_truth, predictions
ground_truth, predictions = extract_ground_truth_and_predictions(results_with_correct_answer)


In [93]:
def calculate_NMI(ground_truth, predictions):
    
    nmi_score = normalized_mutual_info_score(ground_truth, predictions)
    print(f"Normalized Mutual Information Score: {nmi_score}")
    return nmi_score


In [94]:
import numpy as np
from collections import Counter

def calculate_purity(predicted_labels, true_labels):
    # Convert lists to numpy arrays for easier indexing
    predicted_labels = np.array(predicted_labels)
    true_labels = np.array(true_labels)
    
    # Get unique clusters
    unique_clusters = np.unique(predicted_labels)
    
    # Total number of instances
    total_instances = len(true_labels)
    
    # Calculate the number of correctly classified instances in each cluster
    correctly_classified = 0
    for cluster in unique_clusters:
        # Get the true labels of instances in the current cluster
        indices_in_cluster = np.where(predicted_labels == cluster)[0]
        labels_in_cluster = true_labels[indices_in_cluster]
        
        # Determine the most common true label in this cluster
        majority_label_count = Counter(labels_in_cluster).most_common(1)[0][1]
        
        # Add the number of correctly classified instances in this cluster
        correctly_classified += majority_label_count
    
    # Calculate purity
    purity = correctly_classified / total_instances
    print(f"Purity: {purity}")
    return purity





In [95]:
def calculate_accuracy(predicted_labels, true_labels):
    # Ensure that the predicted_labels and true_labels have the same length
    if len(predicted_labels) != len(true_labels):
        raise ValueError("The length of predicted and true labels must be the same.")
    
    # Count the number of correct predictions
    correct_predictions = sum(1 for pred, true in zip(predicted_labels, true_labels) if pred == true)
    
    # Calculate accuracy
    accuracy = correct_predictions / len(true_labels)
    print(f"Accuracy: {accuracy}")
    return accuracy





In [96]:
from sklearn.metrics import f1_score
def calculate_f1_score(ground_truth, predictions):

    f1 = f1_score(ground_truth, predictions, average='micro') # Because there might be over/ under representation of some classes
    print(f"F1 Score: {f1}")
    return f1



In [114]:
calculate_NMI(ground_truth, predictions)
calculate_purity(predictions, ground_truth)
calculate_accuracy(predictions, ground_truth)
calculate_f1_score(ground_truth, predictions)

Normalized Mutual Information Score: 0.6195236098853395
Purity: 0.7596711798839458
Accuracy: 0.7480657640232108
F1 Score: 0.7480657640232108


0.7480657640232108